In [5]:
## In this notebook, we will attempt to find the magnitude of nongravitational 
## acceleration for which heliolinc and link_purify will no longer recognize 
## a given object as a single body. 

In [58]:
## first, import the necessary modules

import os 
import sys
import numpy as np
import pandas as pd
import subprocess
import sorcha

kep_dir = os.path.abspath('/home/ellie/research/lsst/lsst_seti/nongrav/sorcha_ng')
sys.path.insert(0, kep_dir)

from kep2cart import kep2cart as k2c

cm_dir = os.path.abspath('/home/ellie/research/lsst/lsst_seti/nongrav/heliolinc')
sys.path.insert(0, cm_dir)

from create_maketracklets_input import create_maketracklets_input as cmi

In [ ]:
## create the Sorcha input files for one single object 
## the object could be a normal object like Pratchett, 
## or one that already displays nongravs. Favor the former.

## note that there are two ways to approach this overall:

## 1 - Run the ENTIRE PIPELINE for each object with each nongrav setting,
##   - creating different input files each time

## 2 - Figure out a way to determine what object the linkages in a given 
##   - heliolinx output correspond to, whether using orbital element
##   - comparison or some other way. 

## in any case, I believe that the best course of action is to create
## separate input files for each magnitude of nongrav -- i.e., one input 
## file where the A1 magnitude is 10^-6, one input file where the A1 magnitude
## is 10^-5, etc., for each object in the file. 

In [62]:
## create the Sorcha inputs

## first, the orbits file(s)
## read in the Keplerian orbits file first and convert to Cartesian

#k2c("ceres_orb_kep.csv", "ceres_orb_cart.csv")
k2c("pratchett_orb_kep.csv", "/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_cart.csv")

New Cartesian orbits file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_cart.csv written.


In [63]:
## generate the nongravitational orbits files
nongravs = np.logspace(start=-12, stop=3, num=16, base=10) 

df = pd.read_csv("/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_cart.csv")
df['FORMAT'] = 'NONGRAV'
df['a1'] = 0.0
df['a2'] = 0.0
df['a3'] = 0.0
df['model'] = 'ASTEROID'
epoch = df.pop('epochMJD_TDB')
df.insert(12, 'epochMJD_TDB', epoch)

orb_files = ["/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_cart.csv"]

for i in range(len(nongravs)):
    df1 = df.copy()
    df1['a1'] = nongravs[i]
    df1_fname = "/home/ellie/research/lsst/sorcha_output/ceres/pratchett_orb_ng_a1_{}.csv".format(np.log10(nongravs[i]))
    df1.to_csv(df1_fname, index=False)

    df2 = df.copy()
    df2['a2'] = nongravs[i]
    df2_fname = "/home/ellie/research/lsst/sorcha_output/ceres/pratchett_orb_ng_a2_{}.csv".format(np.log10(nongravs[i]))
    df2.to_csv(df2_fname, index=False)

    df3 = df.copy()
    df3['a3'] = nongravs[i]
    df3_fname = "/home/ellie/research/lsst/sorcha_output/ceres/pratchett_orb_ng_a3_{}.csv".format(np.log10(nongravs[i]))
    df3.to_csv(df3_fname, index=False)

    orb_files.append(df1_fname)
    orb_files.append(df2_fname)
    orb_files.append(df3_fname)    

In [13]:
## generate Physical Parameters file using placeholder values
## (these are not the correct values for Ceres, need to correct)

df_phy = pd.DataFrame()
df_phy['ObjID'] = ['ceres']
df_phy['H_r'] = [5.63]
df_phy['u-r'] = [2.55]
df_phy['g-r'] = [0.92]
df_phy['i-r'] = [-0.38]
df_phy['z-r'] = [-0.59]
df_phy['y-r'] = [-0.70]
df_phy['GS'] = [0.15]

df_phy.to_csv("pratchett_phy.csv", index=False)

In [65]:
## run Sorcha

file_id_orb = 'pratchett_orb_cart' #'pratchett_orb_ng_a1_0.0' #'ceres_orb_cart'
file_id = 'pratchett_cart' #'pratchett_ng_a1_0.0'

subprocess.run(["sorcha","run","-c","Rubin_full_footprint.ini","-p","pratchett_phy.csv","--orbits",\
                "/home/ellie/research/lsst/sorcha_output/127005pratchett_new/{}.csv".format(file_id_orb),"--pd",\
                "/home/ellie/src/sorcha/sorcha/demo/baseline_v2.0_1yr.db","-o",\
                "/home/ellie/research/lsst/sorcha_output/127005pratchett_new/","-t",\
                "/home/ellie/research/lsst/sorcha_output/127005pratchett_new/"+file_id,\
                "--st","/home/ellie/research/lsst/sorcha_output/127005pratchett_new/{}_stats".format(file_id)])

CompletedProcess(args=['sorcha', 'run', '-c', 'Rubin_full_footprint.ini', '-p', 'pratchett_phy.csv', '--orbits', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_orb_cart.csv', '--pd', '/home/ellie/src/sorcha/sorcha/demo/baseline_v2.0_1yr.db', '-o', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/', '-t', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart', '--st', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart_stats'], returncode=0)

In [66]:
## shorten the Sorcha output to one month of detections, as heliolinc seems to balk
## at anything much longer, according to Aren

df_sorcha = pd.read_csv("/home/ellie/research/lsst/sorcha_output/127005pratchett_new/{}.csv".format(file_id))
start_date = df_sorcha['fieldMJD_TAI'][0]
end_date = start_date+30

df_sorcha_30d = df_sorcha[df_sorcha['fieldMJD_TAI'] <= end_date]

df_sorcha_30d.to_csv("/home/ellie/research/lsst/sorcha_output/127005pratchett_new/{}_30days.csv".format(file_id), index=False)

In [69]:
## run create_maketracklets_input to create an input file from the Sorcha
## output that is compatible with - you guessed it - make_tracklets

fname = "/home/ellie/research/lsst/sorcha_output/127005pratchett_new/{}_30days.csv".format(file_id)
ofname = "/home/ellie/research/lsst/sorcha_output/127005pratchett_new/{}_30days_cmi.csv".format(file_id)

cmi(fname, ofname)

In [72]:
## run make_tracklets

subprocess.run(['make_tracklets', '-dets', ofname, '-earth', '/home/ellie/research/lsst/heliolinc_files/Earth1day2020s_02a.csv', \
               '-obscode', '/home/ellie/research/lsst/heliolinc_files/ObsCodes.html', '-colformat', \
               '/home/ellie/research/lsst/heliolinc_files/colformat.txt','-outimgs', \
               '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_{}.txt'.format(file_id), \
               '-pairdets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_{}.csv'.format(file_id),\
               '-tracklets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_{}.csv'.format(file_id), \
               '-trk2det', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/trk2det_{}.csv'.format(file_id)])

Checking out argv[1] = -dets.
Checking out argv[3] = -earth.
Checking out argv[5] = -obscode.
Checking out argv[7] = -colformat.
Checking out argv[9] = -outimgs.
Checking out argv[11] = -pairdets.
Checking out argv[13] = -tracklets.
Checking out argv[15] = -trk2det.

Input detection file is called /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart_30days_cmi.csv

No input image file specified: image catalog will be generated internally.
column formatting file = /home/ellie/research/lsst/heliolinc_files/colformat.txt
Observatory code file /home/ellie/research/lsst/heliolinc_files/ObsCodes.html
Output image file will be called /home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_pratchett_cart.txt
Output paired detection file will be called /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv
Output tracklet file will be called /home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_pratchett_cart.csv
O

CompletedProcess(args=['make_tracklets', '-dets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart_30days_cmi.csv', '-earth', '/home/ellie/research/lsst/heliolinc_files/Earth1day2020s_02a.csv', '-obscode', '/home/ellie/research/lsst/heliolinc_files/ObsCodes.html', '-colformat', '/home/ellie/research/lsst/heliolinc_files/colformat.txt', '-outimgs', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_pratchett_cart.txt', '-pairdets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv', '-tracklets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_pratchett_cart.csv', '-trk2det', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/trk2det_pratchett_cart.csv'], returncode=0)

an 14 22:45:52 2026

merge_pairs is sanity-checking indvecs
Sanity-check finished
merge_pairs3 finished OK
Output image catalog /home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_pratchett_cart.txt, with 18 lines, has been written
Writing paired detection file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv with 15 lines
Writing tracklet file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_pratchett_cart.csv with 5 lines
Writing trk2det file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/trk2det_pratchett_cart.csv with 15 lines


In [74]:
## run heliolinx

subprocess.run(['heliolinc', '-imgs', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_{}.txt'.format(file_id), \
                '-pairdets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_{}.csv'.format(file_id), \
                '-tracklets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_{}.csv'.format(file_id), \
                '-trk2det', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/trk2det_{}.csv'.format(file_id), \
                '-obspos', '/home/ellie/research/lsst/heliolinc_files/Earth1day2020s_02a.csv', '-heliodist', \
                '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/hihyp00b_mb.txt', '-outsum',\
                '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/sum_{}.csv'.format(file_id),\
                '-clust2det', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/clust2det_{}.csv'.format(file_id)])

Checking out argv[1] = -imgs.
Checking out argv[3] = -pairdets.
Checking out argv[5] = -tracklets.
Checking out argv[7] = -trk2det.
Checking out argv[9] = -obspos.
Checking out argv[11] = -heliodist.
Checking out argv[13] = -outsum.
Checking out argv[15] = -clust2det.
input image file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_pratchett_cart.txt
input detection file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv
input tracklet file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_pratchett_cart.csv
input trk2det file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/trk2det_pratchett_cart.csv
input observer position file /home/ellie/research/lsst/heliolinc_files/Earth1day2020s_02a.csv
input heliocentric hypothesis file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/hihyp00b_mb.txt
input reference MJD 0
Defaulting to cluster radius = 100000km
Defaulting to min. geocentric distance

CompletedProcess(args=['heliolinc', '-imgs', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_pratchett_cart.txt', '-pairdets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv', '-tracklets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/tracklets_pratchett_cart.csv', '-trk2det', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/trk2det_pratchett_cart.csv', '-obspos', '/home/ellie/research/lsst/heliolinc_files/Earth1day2020s_02a.csv', '-heliodist', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/hihyp00b_mb.txt', '-outsum', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/sum_pratchett_cart.csv', '-clust2det', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/clust2det_pratchett_cart.csv'], returncode=0)

ter distance is 0.23 AU
Geocentric distance step 4, bin-center distance is 0.34 AU
Geocentric distance step 5, bin-center distance is 0.51 AU
Geocentric distance step 6, bin-center distance is 0.76 AU
Geocentric distance step 7, bin-center distance is 1.14 AU
Geocentric distance step 8, bin-center distance is 1.71 AU
KDRclust_6i01 finished clustering geobin 8, with 0 = 0 clusters found
Final analysis of geobin 8 identified 0 distinct candidate linkages. Current total is 0
Geocentric distance step 9, bin-center distance is 2.56 AU
KDRclust_6i01 finished clustering geobin 9, with 0 = 0 clusters found
Final analysis of geobin 9 identified 0 distinct candidate linkages. Current total is 0
Geocentric distance step 10, bin-center distance is 3.84 AU
Geocentric distance step 11, bin-center distance is 5.77 AU
Geocentric distance step 12, bin-center distance is 8.65 AU
Geocentric distance step 13, bin-center distance is 12.97 AU
Geocentric distance step 14, bin-center distance is 19.46 AU
Geoc

In [79]:
## run link_purify -- first, create the lflist file

with open('/home/ellie/research/lsst/sorcha_output/127005pratchett_new/{}_lflist'.format(file_id), 'w') as f:
    f.write("/home/ellie/research/lsst/sorcha_output/127005pratchett_new/sum_{0}.csv /home/ellie/research/lsst/sorcha_output/127005pratchett_new/clust2det_{0}.csv".format(file_id))

In [81]:
## now, actually run link_purify

subprocess.run(['link_purify','-imgs','/home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_{}.txt'.format(file_id),\
                '-pairdets','/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_{}.csv'.format(file_id), \
                '-lflist','/home/ellie/research/lsst/sorcha_output/127005pratchett_new/{}_lflist'.format(file_id),\
                '-outsum','/home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLsum_{}.csv'.format(file_id),\
                '-clust2det','/home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLclust2det_{}.csv'.format(file_id)])

Checking out argv[1] = -imgs.
Checking out argv[3] = -pairdets.
Checking out argv[5] = -lflist.
Checking out argv[7] = -outsum.
Checking out argv[9] = -clust2det.
input paired detection file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv
input cluster list file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart_lflist
Maximum RMS in km: 100000
Maximum astrometric RMS: 1
Maximum fraction of points to be rejected: 0.5
Minimum number of observing nights: 3
Minimum number of unique detections: 6
output cluster file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLclust2det_pratchett_cart.csv
output rms file /home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLsum_pratchett_cart.csv
Reference MJD will be set to MJD-at-the-epoch from a previous orbit fit, if available.
Defaulting to simplex type = 1
Maximum fraction of points that can be rejected = 0.5, which is the default.
Maximum number of points that

CompletedProcess(args=['link_purify', '-imgs', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/outim_pratchett_cart.txt', '-pairdets', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pairdets_pratchett_cart.csv', '-lflist', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/pratchett_cart_lflist', '-outsum', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLsum_pratchett_cart.csv', '-clust2det', '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLclust2det_pratchett_cart.csv'], returncode=0)

In [82]:
## how many linkages?

purify_output = '/home/ellie/research/lsst/sorcha_output/127005pratchett_new/LPLsum_{}.csv'.format(file_id)
df_links = pd.read_csv(purify_output)

print(len(df_links))

1
